# Phonopy and ASE Tutorial

In this tutorial you will learn how to run an automated calculation of phonons by calculating the forces using ASE.

We will use the `aiida_phonopy.workflows.ase.PhonopyAseWorkChain`, which is based on `aiida_pythonjob`.
This workflow takes care of: 

* Pre-processing: generating the (supercell) structures with displacements on top of which computing forces (for frozen phonons)
* Gather all information in `PhonopyData`, ready to be post-processed 
* (optional) Post-process: calculate phonon-related properties, such as phonon band structure and (P)DOS, thermal properties, and so on by using `PhonopyCalculation`.

In this tutorial we will make use of the silicon structure to give you an overall understanding of the usage.

Let's get started!

In [1]:
from local_module import load_temp_profile
from aiida.plugins import DataFactory, WorkflowFactory

# If you download this file, you can run it with your own profile.
# Put these lines instead:
# from aiida import load_profile
# load_profile()
load_temp_profile(
    name="ase-tutorial",
    add_computer=True,
    add_phonopy_code=True,
)

StructureData = DataFactory("core.structure")
PhonopyAseWorkChain = WorkflowFactory("phonopy.ase")

/opt/conda/lib/python3.10/site-packages/aiida/orm/nodes/data/code/legacy.py:64: AiidaDeprecationWarning: The `Code` class is deprecated. To create an instance, use the `aiida.orm.nodes.data.code.installed.InstalledCode` or `aiida.orm.nodes.data.code.portable.PortableCode` for a "remote" or "local" code, respectively. If you are using this class to compare type, e.g. in `isinstance`, use `aiida.orm.nodes.data.code.abstract.AbstractCode`. (this will be removed in v3)
  warn_deprecation(
/opt/conda/lib/python3.10/site-packages/aiida/orm/nodes/data/code/legacy.py:77: AiidaDeprecationWarning: The `Code` plugin is deprecated, use the `InstalledCode` (`core.code.remote`) instead. (this will be removed in v3)
  warn_deprecation(
/opt/conda/lib/python3.10/site-packages/aiida/orm/nodes/data/code/legacy.py:487: AiidaDeprecationWarning: `Code.set_remote_computer_exec` method is deprecated, use `InstalledCode`. (this will be removed in v3)
  warn_deprecation('`Code.set_remote_computer_exec` method 

Let's define the alumin structure using the ASE module

In [2]:
from ase.build import bulk

atoms = bulk("Al", a=1.5456658) # Note: this is NOT the experimental lattice constant, but the good value for the force field used in the tutorial
structure = StructureData(ase=atoms)

```{code-block} python
:caption: |
:    If you have your own structure, e.g., 
:    in .cif or .xyz format, you can simply use the followig snippet

from ase.io import read

atoms = read("/path/to/file.cif") # here, any format supported by ASE
structure = StructureData(ase=atoms)
```

## Automated calculation via `PhonopyAseWorkChain`

We now want to choose an ASE `calculator` that we want to give to the workchain, so that it will compute with it all the forces on all the displaced structures. The calculation will be performed automatically by the `PhonopyAseWorkChain`, which will give us a `PhonopyData` as output that stores all the displacements and forces needed to compute phonons-related properties.

The calculator can be:
- a simple empirical forces field, like a Lenard-Jones potential (presented in this example)
- a DFT calculator (e.g., Quantum ESPRESSO, VASP, Abinit, and so on, interfaced by ASE)
- a pre-trained machine-learning potential (e.g., NequIP, Allegro, MACE, MatterSim, GAP, FLARE, and so on, interfaced with ASE)

These should be installed and able to run on your machine or on a remote cluster.

In [ ]:
from ase.calculators.lj import LennardJones
from aiida.orm import Dict, load_code
from aiida.engine import run_get_node

inputs = PhonopyAseWorkChain.get_populated_builder(
    structure=structure,
    calculator=LennardJones(),
    max_number_of_atoms=200,
    pythonjob_inputs={"computer": "local_direct"},
    phonopy_inputs={
        "code": load_code("phonopy@local_direct"),
        "parameters": Dict({"band":"auto"})
    },
)

results, node = run_get_node(PhonopyAseWorkChain, **inputs)

/aiida-plugins/aiida-phonopy/src/aiida_phonopy/workflows/ase.py:104: DeprecationWarning: dict interface is deprecated. Use attribute interface instead
  to_conv = spglib_dataset['transformation_matrix']  # to conventional cell
07/15/2025 08:10:30 AM <18836> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [13|PhonopyAseWorkChain|run_forces]: submitting `PythonJob` <PK=19> with supercell n.o 1
07/15/2025 08:10:36 AM <18836> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [13|PhonopyAseWorkChain|run_phonopy]: submitting `PhonopyCalculation` <PK=28>
/opt/conda/lib/python3.10/site-packages/aiida/orm/nodes/data/code/legacy.py:556: AiidaDeprecationWarning: `Code.is_local` method is deprecated, use a `PortableCode` instance and check the type. (this will be removed in v3)
  warn_deprecation(
/opt/conda/lib/python3.10/site-packages/aiida/orm/nodes/data/code/legacy.py:517: AiidaDeprecationWarning: `Code.get_remote_computer` method is deprecated, use the `com

KilledError: Process was killed because the runner received an interrupt

07/15/2025 08:15:41 AM <18836> aiida.engine.transports: [ERROR] Exception whilst using transport:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/aiida/engine/transports.py", line 106, in request_transport
    yield transport_request.future
  File "/opt/conda/lib/python3.10/site-packages/aiida/engine/processes/calcjobs/tasks.py", line 372, in do_stash
    return await execmanager.stash_calculation(node, transport)
  File "/opt/conda/lib/python3.10/site-packages/aiida/engine/daemon/execmanager.py", line 455, in stash_calculation
    target_base = Path(stash_options['target_base'])
KeyError: 'target_base'

07/15/2025 08:15:41 AM <18836> aiida.orm.nodes.process.calculation.calcjob.CalcJobNode: [ERROR] iteration 5 of do_stash excepted
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/aiida/engine/utils.py", line 205, in exponential_backoff_retry
    result = await coro()
  File "/opt/conda/lib/python3.10/site-packages/aiid

In [11]:
from aiida.orm import load_node
node = load_node(13)
c = load_node(28)
node.called

[<CalcFunctionNode: uuid: f84944ab-53c9-45b7-8fe6-11a74d808fa9 (pk: 14) (aiida_phonopy.calculations.functions.data_utils.generate_preprocess_data)>,
 <CalcFunctionNode: uuid: 5f5ef796-7dcf-4b52-859c-eaf3c884e4e4 (pk: 16) (aiida_phonopy.calculations.functions.data_utils.get_supercells_with_displacements)>,
 <CalcJobNode: uuid: 49c9059b-ec48-42bd-8e9b-786f1975478e (pk: 19) (aiida.calculations:pythonjob.pythonjob)>,
 <CalcFunctionNode: uuid: b8a74924-4366-4897-afe8-e68789463d93 (pk: 24) (aiida_phonopy.workflows.ase.get_forces_array)>,
 <CalcFunctionNode: uuid: 636c4640-34ca-4a7a-9e1b-fd012f3b751d (pk: 26) (aiida_phonopy.calculations.functions.data_utils.generate_phonopy_data)>,
 <CalcJobNode: uuid: ab9bfa86-9ed6-4e1c-ab94-07c70a00e2a8 (pk: 28) (aiida.calculations:phonopy.phonopy)>]

In [17]:
c.inputs._get_keys()

OperationalError: (sqlite3.OperationalError) no such table: db_dbnode
[SQL: SELECT db_dbnode_1.id, db_dbnode_1.uuid, db_dbnode_1.node_type, db_dbnode_1.process_type, db_dbnode_1.label, db_dbnode_1.description, db_dbnode_1.ctime, db_dbnode_1.mtime, db_dbnode_1.attributes, db_dbnode_1.extras, db_dbnode_1.repository_metadata, db_dbnode_1.dbcomputer_id, db_dbnode_1.user_id, db_dblink_1.type, db_dblink_1.label AS label_1 
FROM db_dbnode AS db_dbnode_2 JOIN db_dblink AS db_dblink_1 ON db_dblink_1.output_id = db_dbnode_2.id JOIN db_dbnode AS db_dbnode_1 ON db_dblink_1.input_id = db_dbnode_1.id 
WHERE CAST(db_dbnode_2.node_type AS VARCHAR) LIKE ? ESCAPE '\' AND db_dbnode_2.id = ? AND CAST(db_dbnode_1.node_type AS VARCHAR) LIKE ? ESCAPE '\' AND db_dblink_1.type IN (?)]
[parameters: ('%', 28, '%', 'input_calc')]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [ ]:
results['output_phonopy']['phonon_bands'].show_mpl()

## Manual post-processing

You can still of course use the output `PhonopyData` to get a `Phonopy` instance, allowing you to directly post-process the data locally.

In [ ]:
ph = node.outputs.phonopy_data.get_phonopy_instance()
ph.produce_force_constants()
ph.auto_band_structure(plot=True)